In [ ]:
#! pip install speedtest-cli

In [ ]:
from datetime import datetime, timedelta
import time
import speedtest
import csv

In [ ]:
STATUS_OUTPUT_FILE = './output/status.txt'  # File to output script status
DATA_FILE = './output/output_file.csv'  # File to store .csv data
SECONDS_BETWEEN_LOOPS = 1  # Interval between checking time slots (not for testing itself)


In [ ]:
# Outputs current status to txt file
def write_status(output_file, output):
    with open(output_file, 'a') as writer:
        writer.write(output)

In [ ]:
# Run the test_speed() function at every quarter of the hour precisely.
def timed_sample_output():
    times = ['00', '15', '30', '60']  # List of quarter-hours

    while True:  # Loop to continually test current time against quarter-hours
        current_time = datetime.now().strftime('%H:%M')
        for quarter in times:
            if current_time[3:5] == quarter:
                test_speed(current_time)
                # Sleep until the next quarter-hour time, so we don't test too frequently
                next_quarter_time = (datetime.now() + timedelta(minutes=15)).replace(second=0, microsecond=0)
                sleep_time = (next_quarter_time - datetime.now()).total_seconds()
                time.sleep(sleep_time)
                break
        time.sleep(SECONDS_BETWEEN_LOOPS)


In [ ]:
def test_speed(curr_time):
    """Takes a sample of internet connection Down/Up speeds and puts the results in a dictionary.
    Outputs the results of each test onto console."""
    SPEED_DIVISOR = 1000000  # for converting bits per second into Mbps

    s = speedtest.Speedtest()
    try:
        download_time = s.download()
        upload_time = s.upload()
    except Exception as e:
        write_status(STATUS_OUTPUT_FILE, f"Error during speed test at {curr_time}: {e}\n")
        return

    today = datetime.now().date()
    output = {
        "Date": today,
        "Time": curr_time,
        "Download": round((download_time / SPEED_DIVISOR), 2),
        "Upload": round((upload_time / SPEED_DIVISOR), 2)
    }
    write_status(STATUS_OUTPUT_FILE, f"Trial for time: {curr_time} completed\n{output}\n")
    write_to_file(output)


In [ ]:
# Takes a test conducted by test_speed() and stores it in a CSV file.
def write_to_file(output):
    """Write the result of the speed test to a CSV file."""
    try:
        with open(DATA_FILE, 'a', newline='') as write_file:
            columns = ['Date', 'Time', 'Download', 'Upload']
            writer = csv.DictWriter(write_file, fieldnames=columns)
            # Write the header only once when the file is created (if needed)
            if write_file.tell() == 0:  # Check if file is empty
                writer.writeheader()
            writer.writerow(output)
    except Exception as e:
        write_status(STATUS_OUTPUT_FILE, f"Error writing to CSV: {e}\n")

In [ ]:
'''Run the program!'''
if __name__ == "__main__":
    timed_sample_output()

KeyboardInterrupt: 